# The Solow Model with limited land

Imports and set magics:

In [20]:
import numpy as np
from scipy import optimize
import sympy as sm

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
#import modelproject

# Import our class
#from modelproject import modelprojectclass

# We now define our model 
#model = modelprojectclass()

#par = model.par

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Model description

**The Solov Model with land**  

The model is described as:

1. $K_t$ is capital2
2. $L_t$ is labor (growing with a constant rate of $n$)
3. $A_t$ is technology (growing with a constant rate of $g$)
4. $X$ is land which is a non-changing input, therefore it does not depend on the time period 
5. $Y_t = F(K_t,A_tL_t, X)$ is GDP

**GDP** is given as
$$ 
Y_t = K_t^\alpha(A_t L_t)^\beta X^\kappa \\
 \alpha > 0, \beta > 0, \kappa > 0, \alpha + \beta + \kappa = 1 
$$

The **production function** has **decreasing-return to scale** depending on the value of $\kappa$

**Saving** is a constant fraction of GDP
$$ 
S_t = sY_t,\,s\in(0,1)
$$

such that **capital accumulates** according to

$$
K_{t+1}=S_{t}+(1-\delta)K_{t}, \delta\in(0,1)
$$

**Labor** is assumed to be the whole population
$$
L_{t+1}=(1 + n) L_t, n \geq 0
$$

**Technology** is described as
$$
A_{t+1}= (1 + g) A_t, g \geq 0
$$



## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

In order to solve the model, we need to define the capital-output ratio, $z_t$, which is given as: 
$$
z_t = K_t / Y_t = k_t / y_t
$$

The small letters represents per capita terms. 

This means that the per capita production function is: 
$$
y_t = k_t^\alpha A_t^\beta x_t^\kappa
$$

Therefore $z_t$ can also be described as: 
$$
z_t = k_t^{1-\alpha} A_t^{-\beta} x_t^{-\kappa}
$$

We want to find the steady state function, we therefore use **sympy** to find an analytical expression for the steady state, i.e. we solve the transition equation for $z_t$:
$$
z_{t+1} = (\frac{1}{(1+n)(1+g)})^\beta [s+(1-\delta) z_t]^{1-\alpha} z_t^{\alpha}
$$

In the steady state the capital output ratio has found a fixed value depending on the variables. We therefore get $z_{t+1} = z_t = z^*$

In [21]:
# We start out by defining the symbols
z = sm.symbols('z')
alpha = sm.symbols('alpha')
delta = sm.symbols('delta')
beta = sm.symbols('beta')
kappa = sm.symbols('kappa')
s = sm.symbols('s')
g = sm.symbols('g')
n = sm.symbols('n')
x = sm.symbols('x')
A = sm.symbols('A')
k = sm.symbols('k')

In [22]:
# write your code here
z = k**(1-alpha)*A**(-beta)* x**(-kappa)

# k, da vi løser for k
ss = sm.Eq(z, ((1/((1+n)*(1+g)))**beta*(s+(1-delta)*z)**(1-alpha)*z**alpha))

# løser ss for k, kun en løsning derfor 0 
zss = sm.solve(ss,z)[0]
zss

NotImplementedError: multiple generators [(-_X0*delta + _X0 + s)**(1 - alpha), _X0, _X0**alpha]
No algorithms are implemented to solve equation -_X0**alpha*(1/((g + 1)*(n + 1)))**beta*(_X0*(1 - delta) + s)**(1 - alpha) + _X0

In [ ]:
# write your code here
ss_func = sm.lambdify((s,g,n,delta,alpha),kss)

# Evaluate function
ss_func(0.2,0.02,0.01,0.1,1/3)

NameError: name 'kss' is not defined

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 